![ARL](https://miro.medium.com/v2/resize:fit:532/0*d4HGOpwY6z2x1Rtk)

# What is Association Rule Learning

- It is a rule-based machine learning technique used to find patterns in data. The Apriori Algorithm is used while the Association Rule Learning takes place. Apriori is a basket analysis method used to reveal product associations. 

- **There are 3 significant metrics in Apriori:**

- **Support:** Measures how often products X and Y are purchased together
- **Support(X, Y) = Freq(X, Y) / Total Transaction**

- **Confidence:** Probability of purchasing product Y when product X is purchased
- **Confidence(X, Y) = Freq(X, Y) / Freq(X)**

- **Lift:** The coefficient of increase in the probability of purchasing product Y when product X is purchased.
- **Lift = Support(X, Y) / (Support(X) * Support(Y))**

# Business Problem

- We want to create a product recommendation system with Association Rule Learning using a dataset containing service users and the services and categories that these users have received.



# Data Story

- The data set consists of the services received by customers and the categories of these services.
- It contains the date and time information of each service received.

- **UserId:** Customer number
- **ServiceId:** Anonymized services belonging to each category (Example: Sofa washing service under the cleaning category)
- A ServiceId can be found under different categories and refers to different services under different categories. (Example: CategoryId 7 ServiceId 4  refers to honeycomb cleaning while CategoryId 2 ServiceId 4 refers to furniture assembly)
- **CategoryId:** Anonymized categories (Example: Cleaning, transportation, renovation category)
- **CreateDate:** Date the service was purchased

# Road Map

- **1.** Data Preprocessing
- **2.** Preparing the ARL Data Structure 
- **3.** Extraction of Association Rules
- **4.** Suggesting Products to Users at the Cart Stage

In [1]:
!pip install mlxtend

In [2]:
# import Required Libraries

import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

# 1. Data Preprocessing

In [4]:
# Loading the Data Set

df_ = pd.read_csv("/kaggle/input/armut-data/armut_data.csv")

In [5]:
df = df_.copy()

In [6]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [7]:
# Preliminary examination of the data set

def check_df(dataframe, head=5):
    print('##################### Shape #####################')
    print(dataframe.shape)
    print('##################### Types #####################')
    print(dataframe.dtypes)
    print('##################### Head #####################')
    print(dataframe.head(head))
    print('##################### Tail #####################')
    print(dataframe.tail(head))
    print('##################### NA #####################')
    print(dataframe.isnull().sum())
    print('##################### Quantiles #####################')
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(162523, 4)
##################### Types #####################
UserId         int64
ServiceId      int64
CategoryId     int64
CreateDate    object
dtype: object
##################### Head #####################
   UserId  ServiceId  CategoryId           CreateDate
0   25446          4           5  2017-08-06 16:11:00
1   22948         48           5  2017-08-06 16:12:00
2   10618          0           8  2017-08-06 16:13:00
3    7256          9           4  2017-08-06 16:14:00
4   25446         48           5  2017-08-06 16:16:00
##################### Tail #####################
        UserId  ServiceId  CategoryId           CreateDate
162518   10591         25           0  2018-08-06 14:40:00
162519   10591          2           0  2018-08-06 14:43:00
162520   10591         31           6  2018-08-06 14:47:00
162521   12666         38           4  2018-08-06 16:01:00
162522   17497         47           7  2018-08-06 16:04:00
##############

# 2. Preparing the ARL Data Structure (Invoice-Product Matrix)

In [8]:
# We created a new variable to represent services by combining ServiceID and CategoryID with "_"

df["Hizmet"] = [str(row[1]) + "_" + str(row[2]) for row in df.values]

In [9]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


- The data set consists of the date and time when the services were received, there is no basket definition (invoice etc.). In order to apply Association Rule Learning, a basket (invoice etc.) definition must be created. Here, the basket definition is the monthly services received by each customer. For example; 9_4, 46_4 services received by the customer with id 7256 in the 8th month of 2017 represent one basket; 9_4, 38_4 services received in the 10th month of 2017 represent another basket. Baskets should be identified with a unique ID. For this, we first created a new date variable containing only year and month. We assigned the UserID and the date variable you just created to a new variable called ID by combining it with "_".

In [10]:
df["CreateDate"] = pd.to_datetime(df["CreateDate"])

In [11]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [12]:
df["NEW_DATE"] = df["CreateDate"].dt.strftime("%Y-%m")

In [13]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,NEW_DATE
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08


In [14]:
df["SepetID"] = [str(row[0]) + "_" + str(row[5]) for row in df.values]

In [15]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,NEW_DATE,SepetID
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08


# 3. Extraction of Association Rules

In [16]:
# Invoice-Product Matrix created

invoice_product_df = df.groupby(['SepetID', 'Hizmet'])['Hizmet'].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

In [17]:
invoice_product_df.head()

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [18]:
# We obtained all probabilities by creating association rules with apriori.

frequent_itemsets = apriori(invoice_product_df, min_support=0.01, use_colnames=True)

/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [19]:
frequent_itemsets.head()

,support,itemsets
0,0.019728,(0_8)
1,0.026523,(11_11)
2,0.029374,(12_7)
3,0.056627,(13_11)
4,0.023406,(14_7)


In [20]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

In [21]:
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216,0.449965
1,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325,0.488074
2,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066,0.609539
3,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833,0.616073
4,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262,0.803047
5,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211,0.725728
6,(38_4),(15_1),0.066568,0.120963,0.011177,0.167897,1.388001,0.003124,1.056404,0.299475
7,(15_1),(38_4),0.120963,0.066568,0.011177,0.092397,1.388001,0.003124,1.028458,0.318007
8,(15_1),(49_1),0.120963,0.067762,0.010011,0.082763,1.221375,0.001815,1.016354,0.206192
9,(49_1),(15_1),0.067762,0.120963,0.010011,0.147741,1.221375,0.001815,1.031420,0.194425


# 4. Suggesting Products to Users at the Cart Stage

In [22]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = [] 
    for i, product in sorted_rules["antecedents"].items():
        for j in list(product): 
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))
    recommendation_list = list({item for item_list in recommendation_list for item in item_list})
    return recommendation_list[:rec_count] 

In [23]:
arl_recommender(rules,"2_0", 4)

['38_4', '13_11', '22_0', '15_1']

In [24]:
arl_recommender(rules,"13_11", 4)

['25_0']